In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Instala (se ainda não instalou)
!pip install -q tensorflow-datasets

# Carrega bibliotecas
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

# Verificação da versão
print("TensorFlow version:", tf.__version__)

# =======================================================
# 1) Carregar o dataset cats_vs_dogs diretamente do TFDS
# =======================================================
# 'cats_vs_dogs' tem todas as imagens como um único split 'train'.
# Vamos dividir em 80% treino e 20% validação aqui.
(ds_full, ), ds_info = tfds.load('cats_vs_dogs', split=['train'], with_info=True, as_supervised=True)

# Obtem o total de exemplos para calcular splits
total_examples = ds_info.splits['train'].num_examples
print("Total imagens disponíveis:", total_examples)

# Criar splits manualmente: 80% treino, 20% validação
train_size = int(0.8 * total_examples)
val_size = total_examples - train_size

ds = ds_full.shuffle(1000, seed=123)

ds_train = ds.take(train_size)
ds_val = ds.skip(train_size)

# =======================================================
# 2) Função de pré-processamento (resize + normalizar)
# =======================================================
IMG_HEIGHT = 150
IMG_WIDTH = 150

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

BATCH_SIZE = 32

train_ds = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = ds_val.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Conjuntos prontos:")
print(" - train batches:", len(list(train_ds)))
print(" - val batches:", len(list(val_ds)))

TensorFlow version: 2.19.0


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.0Z8XJE_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Total imagens disponíveis: 23262
Conjuntos prontos:
 - train batches: 582
 - val batches: 146


In [2]:
# ===============================================
# Criar o modelo base pré-treinado (MobileNetV2)
# ===============================================

IMG_SHAPE = (150, 150, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

# Congelar a base para o primeiro treinamento
base_model.trainable = False

# ===============================================
# Criar o topo do modelo (igual ao notebook)
# ===============================================

global_avg_pooling = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(1)  # binário: 0/1 gatos vs cachorros

inputs = keras.Input(shape=IMG_SHAPE)
x = base_model(inputs, training=False)
x = global_avg_pooling(x)
outputs = prediction_layer(x)

model = keras.Model(inputs, outputs)

model.summary()

/tmp/ipython-input-2649607438.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [4]:
initial_epochs = 5

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs
)

NameError: name 'train_ds' is not defined

In [5]:
base_model.trainable = True

In [6]:
print("Treináveis:", len(base_model.trainable_variables))

Treináveis: 156


In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [8]:
fine_tune_epochs = 5

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs
)

NameError: name 'train_ds' is not defined

In [9]:
model.save("modelo_transfer_learning.h5")